[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Svetlana-Yatsyk/JdE_RMBLF_texto/blob/main/notebooks/lemmatisation.ipynb)

Pour obtenir le résultat, il vous suffit d’exécuter successivement **toutes les cellules** de ce notebook.<br><br>
Pour cela, cliquez sur le **bouton «lecture»** à gauche de chaque cellule, ou utilisez le raccourci **Maj + Entrée**.<br><br>
Il est très important de **respecter l’ordre d’exécution**, en allant de la première cellule jusqu’à la dernière.

# Préparation

Dans la première cellule, nous installons la bibliothèque **`stanza`**. Une bibliothèque est un ensemble de fonctions ou de commandes prêtes à l’emploi.<br><br>
**`Stanza`** est une bibliothèque capable d’effectuer la **lemmatisation** et l’**analyse morpho-syntaxique**, c’est-à-dire l’identification des parties du discours. Elle est [adaptée à de nombreuses langues](https://stanfordnlp.github.io/stanza/performance.html) et, pour le latin, elle propose quatre modèles différents.


In [6]:
!pip install stanza

Nous importons maintenant la bibliothèque **`stanza`** et deux autres bibliothèques.

In [20]:
import stanza
import pandas as pd
from pathlib import Path


# Test

Nous allons maintenant créer une variable contenant quelques versets du deuxième chapitre du Cantique des Cantiques.

In [10]:
canticum = "Sicut lilium inter spinas, sic amica mea inter filias. " \
"Sicut malus inter ligna silvarum, sic dilectus meus inter filios. " \
"Sub umbra illius quem desideraveram sedi, et fructus ejus dulcis gutturi meo."

Cette cellule crée un **pipeline stanza**, c’est-à-dire une chaîne de traitement. <br>
Ici, on indique que la langue est le latin (`lang='la'`) et que l’on utilise le **modèle «perseus»**.<br><br>
Le paramètre `processors` précise les étapes du traitement :

* `tokenize` : découper le texte en mots (*tokens*),
* `pos` : identifier la partie du discours (nom, verbe, adjectif, etc.),
* `lemma` : ramener chaque mot à sa forme de base (lemmatisation),
* `depparse` : analyser les relations syntaxiques entre les mots.

Cela peut prendre un certain temps, car les modèles sont assez volumineux.


In [11]:
nlp_stanza = stanza.Pipeline(lang='la', package="perseus", processors='tokenize, pos, lemma, depparse')

2025-10-26 23:53:18 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-10-26 23:53:18 INFO: Downloaded file to /Users/sy/stanza_resources/resources.json
2025-10-26 23:53:18 WARNING: Language la package perseus expects mwt, which has been added
2025-10-26 23:53:20 INFO: Loading these models for language: la (Latin):
| Processor | Package          |
--------------------------------
| tokenize  | perseus          |
| mwt       | perseus          |
| pos       | perseus_nocharlm |
| lemma     | perseus_nocharlm |
| depparse  | perseus_nocharlm |

2025-10-26 23:53:20 INFO: Using device: cpu
2025-10-26 23:53:20 INFO: Loading: tokenize
2025-10-26 23:53:22 INFO: Loading: mwt
2025-10-26 23:53:22 INFO: Loading: pos
2025-10-26 23:53:25 INFO: Loading: lemma
2025-10-26 23:53:25 INFO: Loading: depparse
2025-10-26 23:53:26 INFO: Done loading processors!


La commande `nlp_stanza(canticum)` lance le traitement : `Stanza` segmente le texte, identifie les catégories grammaticales, attribue à chaque mot sa lemme, puis établit les relations syntaxiques entre eux.<br>
Le résultat est enregistré dans une nouvelle variable appelée `canticum_lemmatised`.

In [14]:
canticum_lemmatised=nlp_stanza(canticum)

Cette cellule parcourt le texte que nous avons analysé avec `stanza` et affiche, pour chaque mot, trois informations :

* **`token.text`** → la forme originale du mot telle qu’elle apparaît dans le texte,
* **`token.lemma`** → sa **lemme**, c’est-à-dire la forme de base du mot,
* **`token.pos`** → sa **catégorie grammaticale** (nom, verbe, adjectif, etc.).

Autrement dit, cette boucle permet de visualiser le résultat de la lemmatisation et de l’analyse morpho-syntaxique pour chaque mot du passage.

In [15]:
for sent in canticum_lemmatised.sentences:
  for token in sent.words:
    print(token.text + ' - ' + token.lemma + ' - ' + token.pos)

Sicut - sicut - SCONJ
lilium - lilius - NOUN
inter - inter - ADP
spinas - spina - NOUN
, - , - PUNCT
sic - sic - ADV
amica - amica - NOUN
mea - meus - DET
inter - inter - ADP
filias - filia - NOUN
. - . - PUNCT
Sicut - sicut - SCONJ
malus - malus - ADJ
inter - inter - ADP
ligna - lignum - NOUN
silvarum - silva - NOUN
, - , - PUNCT
sic - sic - ADV
dilectus - diligo - VERB
meus - meus - DET
inter - inter - ADP
filios - filius - NOUN
. - . - PUNCT
Sub - sub - ADP
umbra - umbra - NOUN
illius - ille - DET
quem - qui - PRON
desideraveram - desidero - VERB
sedi - sedes - NOUN
, - , - PUNCT
et - et - CCONJ
fructus - fructus - NOUN
ejus - is - PRON
dulcis - dulcis - ADJ
gutturi - gunfero - VERB
meo - meus - DET
. - . - PUNCT


# Analyse de la Vulgate (Genesis)

Cette cellule télécharge deux fichiers depuis le dépôt GitHub :

1. **`stopwords_lat.txt`** — une liste de *mots vides* (mots très fréquents comme *et, in, de,* etc.),
   que l’on exclut souvent des analyses statistiques, car ils n’apportent pas de sens lexical propre. Cette liste est créée par [Aurélien Berra](https://github.com/aurelberra/stopwords/tree/master) ;
2. **`vulgata_full_clean.txt`** — le texte complet de la Vulgate nettoyé de ses références de chapitres et de versets.

Ensuite :

* la variable **`stopwords`** contient tous les mots vides sous forme de liste (un mot par ligne) ;
* la variable **`vulgata`** contient tout le texte de la Vulgate sous forme d’une grande chaîne de caractères.

Autrement dit, cette cellule prépare les **données textuelles** et la **liste de mots à ignorer** pour les étapes suivantes de l’analyse.

In [ ]:
!wget https://raw.githubusercontent.com/Svetlana-Yatsyk/JdE_RMBLF_texto/main/data/stopwords_lat.txt
#!wget https://raw.githubusercontent.com/Svetlana-Yatsyk/JdE_RMBLF_texto/main/data/vulgata_full_clean.txt
!wget https://raw.githubusercontent.com/Svetlana-Yatsyk/JdE_RMBLF_texto/main/data/Genesis.txt

stopwords = open("/content/stopwords_lat.txt",'r',encoding="utf8").read().split("\n")
#vulgata = open("/content/vulgata_full_clean.txt",'r',encoding="utf8").read()
genesis = open("/content/Genesis.txt",'r',encoding="utf8").read()

--2025-10-27 00:04:15--  https://raw.githubusercontent.com/Svetlana-Yatsyk/JdE_RMBLF_texto/main/data/stopwords_lat.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31320 (31K) [text/plain]
Saving to: ‘stopwords_lat.txt’

stopwords_lat.txt   100%[===================>]  30,59K  --.-KB/s    in 0,01s   

2025-10-27 00:04:15 (2,20 MB/s) - ‘stopwords_lat.txt’ saved [31320/31320]

--2025-10-27 00:04:16--  https://raw.githubusercontent.com/Svetlana-Yatsyk/JdE_RMBLF_texto/main/data/vulgata_full_clean.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response...

FileNotFoundError: [Errno 2] No such file or directory: '/content/stopwords_lat.txt'

In [17]:
nlp_stanza = stanza.Pipeline(lang='la', processors='tokenize, pos, lemma')

2025-10-27 00:03:21 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-10-27 00:03:22 INFO: Downloaded file to /Users/sy/stanza_resources/resources.json
2025-10-27 00:03:22 WARNING: Language la package default expects mwt, which has been added
2025-10-27 00:03:30 INFO: Loading these models for language: la (Latin):
| Processor | Package       |
-----------------------------
| tokenize  | ittb          |
| mwt       | ittb          |
| pos       | ittb_nocharlm |
| lemma     | ittb_nocharlm |

2025-10-27 00:03:30 INFO: Using device: cpu
2025-10-27 00:03:30 INFO: Loading: tokenize
2025-10-27 00:03:30 INFO: Loading: mwt
2025-10-27 00:03:30 INFO: Loading: pos
2025-10-27 00:03:35 INFO: Loading: lemma
2025-10-27 00:03:35 INFO: Done loading processors!


Cette cellule définit une fonction qui traite le texte par lots («batches») à l’aide du modèle stanza, 
effectue la tokenisation, la lemmatisation et l’analyse morpho-syntaxique,
puis enregistre les résultats dans un fichier CSV contenant pour chaque mot sa forme, sa lemme et sa catégorie grammaticale.

In [ ]:
def batch_process(text, nlp, batch_size=100, output_prefix="genesis"):
    paragraphs = [p for p in text.split('\n') if p.strip()]
    words = []

    for i in range(0, len(paragraphs), batch_size):
        batch_text = '\n'.join(paragraphs[i:i + batch_size])
        doc = nlp(batch_text)
        words.extend(
            {
                "word": w.text,
                "lemma": w.lemma,
                "pos": w.pos
            }
            for s in doc.sentences
            for w in s.words
            if w.lemma
        )

    df = pd.DataFrame(words)
    outdir = Path(".")
    outdir.mkdir(exist_ok=True)

    # Save CSV
    csv_path = outdir / f"{output_prefix}_lemmatized.csv"
    df.to_csv(csv_path, index=False, encoding="utf-8")

    # Create TXTs
    (outdir / f"{output_prefix}_lemmas.txt").write_text(
        "\n".join(df["lemma"].tolist()), encoding="utf-8"
    )

    (outdir / f"{output_prefix}_types.txt").write_text(
        "\n".join(sorted(set(df["lemma"]))), encoding="utf-8"
    )

    (outdir / f"{output_prefix}_tokens.txt").write_text(
        "\n".join(df["word"].tolist()), encoding="utf-8"
    )

    print(" Fichiers enregistrés :")
    print(f" - {csv_path}")
    print(f" - {output_prefix}_lemmas.txt")
    print(f" - {output_prefix}_types.txt")
    print(f" - {output_prefix}_tokens.txt")

    return df


On lance le traitement

In [ ]:
genesis_lem = batch_process(genesis, nlp_stanza)

Cette cellule parcourt la liste des tokens issus de la lemmatisation. Pour chaque mot, elle extrait sa **forme originale** (`form`) et sa **lemme** (`lemma`).
Si la lemme n’est pas un signe de ponctuation, elle est ajoutée aux listes `forms` et `lemmas`.
Enfin, si la lemme n’est ni une ponctuation ni un mot vide (*stopword*), elle est ajoutée à la liste `no_stop`.

In [ ]:
forms = []
lemmas = []
no_stop = []

for token in genesis_lem:
    form = token["word"]
    lemma = token["lemma"]

    if lemma not in string.punctuation:
        forms.append(form)
        lemmas.append(lemma)

    if lemma not in string.punctuation and lemma not in stopwords:
        no_stop.append(lemma)

print("Nombre total de formes :", len(forms))
print("Nombre total de lemmes :", len(lemmas))
print("Nombre de lemmes sans stopwords :", len(no_stop))

Ce code permet de générer les **nuage de mots**.
<br><br><br>
Je remercie **Marianne Reboul** pour ce code, ainsi que pour l’idée générale de ce notebook, qui reprend en grande partie l’un de ses [supports pédagogiques](https://colab.research.google.com/github/OdysseusPolymetis/journees_cluster5b_7/blob/main/3_nlp_lat_gk.ipynb).

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np

def create_word_cloud(words_list, title):
    text = ' '.join(words_list)

    radius = 495

    diameter = radius * 2
    center = radius
    x, y = np.ogrid[:diameter, :diameter]
    mask = (x - center) ** 2 + (y - center) ** 2 > radius ** 2
    mask = 255 * mask.astype(int)

    mask_rgba = np.dstack((mask, mask, mask, 255 - mask))

    wordcloud = WordCloud(repeat=False, width=diameter, height=diameter,
                          background_color=None, mode="RGBA", colormap='plasma',
                          mask=mask_rgba).generate(text)

    plt.figure(figsize=(10, 10))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
create_word_cloud(forms, 'Types')

In [ ]:
create_word_cloud(lemma, 'Lemmas')

In [ ]:
create_word_cloud(no_stop, 'Lemmas sans mots vides')

Cette cellule permet de télécharger les formes et les lemmes obtenus.<br>
Ils pourront ensuite être importés dans Voyant Tools pour une analyse complémentaire.

with open("/content/forms.txt", "w", encoding="utf8") as f, open("/content/lemmas.txt", "w", encoding="utf8") as f2, open("/content/pullito.txt", "w", encoding="utf8") as f3:
    f.write("\n".join(forms))
    f2.write("\n".join(lemmas))
    f3.write("\n".join(no_stop))